In [1]:
import sys

import tensorflow as tf
from tensorflow import keras as kr

from time import sleep
import numpy as np
import random
import pandas as pd
pd.set_option('display.max_rows', 200)

import sklearn as sk

from sklearn.preprocessing import MinMaxScaler

import matplotlib 

import matplotlib.pyplot as plt

import zipfile
import os

import supportFunctions as sf

import inspect
lines = inspect.getsource(sf.highest)

In [2]:
#função utilizada para normalizar os dados de treino e validação
def normalize_data(train, test):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(train)
    train_scaled = scaler.transform(train)
    test_scaled = scaler.transform(test)
    
    return scaler, train_scaled, test_scaled

#função utilizada para desnormalizar os dados, no formato de entrada
def denormalize_data(scaler,data):
    result = scaler.inverse_transform(data)
    return result

#função utilizada para desnomalizar o valor predito
def denormalize_prediction(scaler, dataX, dataY):
    formatted_data = np.array(dataX)
    formatted_data[:,3] = np.array(dataY)
    
    return denormalize_data(scaler,formatted_data)[:,3]

In [3]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")
# tf.debugging.set_log_device_placement(False)

Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]
Pandas 1.5.2
Scikit-Learn 1.2.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT AVAILABLE


In [4]:
#quantos passos para trás os indicadores técnicos vão olhar
ti_memory = 10

#lendo os dados que serão utilizados para treinamento e validação
ge1day = pd.read_csv('Data\GE.csv')
#removendo a coluna que indica a data
ge1day = ge1day.drop(['Date'], axis = 1)

df = ge1day
#reseting the index
df.index = range(len(df))

In [7]:
#adicionando os parametros de indicadores tecnicos ao dataframe
df["""highest_{}""".format(ti_memory)] = sf.highest(df.Close,ti_memory)
df["""lowest_{}""".format(ti_memory)] = sf.lowest(df.Close,ti_memory)
df["""wma_{}""".format(ti_memory)] = sf.wma(df.Close,ti_memory)
df["""ema_{}""".format(ti_memory)] = sf.ema(df.Close,ti_memory)
df["""hma_{}""".format(ti_memory)] = sf.hma(df.Close,ti_memory)
df["""macd_12_26"""] = sf.macd(df.Close,12,26)
df["""rsi_{}""".format(ti_memory)] = sf.rsi(df.Close,ti_memory)
df["""dpo_{}""".format(ti_memory)] = sf.dpo(df.Close,ti_memory)

#parametros nao utilizados:
# df["""sma_{}""".format(ti_memory)] = sf.sma(df.Close,ti_memory)
# df["""so_k_5"""] = sf.so_k(df.Close)
# df["""so_d_3"""] = sf.so_d(df.Close)
# df["""obv""".format(ti_memory)] = sf.obv(df.Close,df.Volume)

C:\Users\guilh\PythonCodes\supportFunctions.py:263: RuntimeWarning: divide by zero encountered in divide
  rs = ag_vector/al_vector #numpy arrays so therefore can use element wise division
C:\Users\guilh\PythonCodes\supportFunctions.py:263: RuntimeWarning: invalid value encountered in divide
  rs = ag_vector/al_vector #numpy arrays so therefore can use element wise division


In [8]:
display(df)

,Open,High,Low,Close,Adj Close,Volume,highest_10,lowest_10,wma_10,ema_10,hma_10,macd_12_26,rsi_10,dpo_10
0,0.751202,0.763722,0.743690,0.748698,0.001782,2156500,0.748698,0.748698,0.748698,0.748698,0.748698,0.000000,NaN,0.000000
1,0.744942,0.744942,0.738682,0.741186,0.001764,1477600,0.748698,0.741186,0.743690,0.744942,0.745359,0.000000,100.000000,-0.744942
2,0.741186,0.747446,0.726162,0.732422,0.001743,1837000,0.748698,0.732422,0.738056,0.740769,0.741708,0.000000,100.000000,-0.740769
3,0.732422,0.733674,0.701122,0.713642,0.001698,2725600,0.748698,0.713642,0.728290,0.733987,0.734112,0.000000,100.000000,-0.733987
4,0.713642,0.713642,0.691106,0.712390,0.001695,3095000,0.748698,0.712390,0.722990,0.729667,0.727268,0.000000,100.000000,-0.729667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14658,7.630000,8.300000,7.510000,8.120000,8.120000,123180900,8.120000,6.110000,7.062182,7.424700,7.239323,-1.116727,50.108961,-0.399000
14659,7.680000,7.870000,7.540000,7.620000,7.620000,93299000,8.120000,6.110000,7.175091,7.460209,7.697586,-1.029444,40.027642,-0.496000
14660,7.540000,7.940000,7.350000,7.890000,7.890000,86850200,8.120000,6.110000,7.341273,7.538353,8.019525,-0.927790,41.907705,-0.579000
14661,7.870000,8.180000,7.820000,7.940000,7.940000,121149900,8.120000,6.110000,7.494182,7.611380,8.165303,-0.833585,32.316173,-1.075000


In [9]:
#jogando alguns dados fora para termos todas variaveis incializadas
inicialization_steps = 30
prediction_ahead = 10

dataValues = df.values

trainSize = int(len(dataValues)*0.8)
testSize = len(dataValues) - trainSize

train = df.head(trainSize)
# train,minDataTrain,minMaxDataTrain = sf.normalizaTrain(train)
test = df.tail(testSize)
# test = sf.normalizaTest(test, minDataTrain, minMaxDataTrain)

scaler, train_normalized, test_normalized = normalize_data(train, test)

#deixando um valor de fora, para podermos prever o próximo valor quando for o último:
trainX = train_normalized[inicialization_steps:(len(train)-prediction_ahead)]
testX = test_normalized[0:(len(test)-prediction_ahead)]

#normalizando as entradas
# scalerX, trainX, testX = normalize_data(trainX, testX)

#pegando apenas o valor que queremos prever
trainY = train_normalized[(inicialization_steps+prediction_ahead):len(train),3]
testY = test_normalized[prediction_ahead:len(test),3]

#calculando a diferenca percentual do preco de fechamento entre um dia e outro
# trainY = (train.values[(inicialization_steps+1):len(train),3]/trainX[:,3])-1
# testY = (test.values[1:len(test),3]/testX[:,3])-1


# scalerY, trainY, testY = normalize_data(trainY, testY)

In [10]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(11690, 14)
(11690,)
(2923, 14)
(2923,)


In [11]:
#checking if it makes sense
# print(trainInputX[:10,(rnn_memory-1),3])
# print(trainInputY[:10])

In [12]:
rnn_memory = 50

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_50 = []
trained_models_rnn_50_history = []

for i in range(0,20):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_rnn_50.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_50[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_50[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_50[i].add(kr.layers.Dense(1))
    trained_models_rnn_50[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_50_history.append(trained_models_rnn_50[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_50[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_'+str(i))

(11640, 50, 14)
(11640,)
(2873, 50, 14)
(2873,)
11690
11640
11690
11640
(11640, 50, 14)
Epoch 1/2000
46/46 [==============================] - 1s 9ms/step - loss: 0.0346 - val_loss: 0.0254
Epoch 2/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0054 - val_loss: 0.0163
Epoch 3/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0028 - val_loss: 0.0117
Epoch 4/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0019 - val_loss: 0.0079
Epoch 5/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0015 - val_loss: 0.0057
Epoch 6/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0012 - val_loss: 0.0044
Epoch 7/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0010 - val_loss: 0.0038
Epoch 8/2000
46/46 [==============================] - 0s 6ms/step - loss: 9.1751e-04 - val_loss: 0.0033
Epoch 9/2000
46/46 [==============================] - 0s 6ms/step - loss: 8.3102e-04 - val_loss: 0.0031
Epoc

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
46/46 [==============================] - 1s 10ms/step - loss: 0.0238 - val_loss: 0.0140
Epoch 2/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0043 - val_loss: 0.0067
Epoch 3/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0027 - val_loss: 0.0043
Epoch 4/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0020 - val_loss: 0.0032
Epoch 5/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0016 - val_loss: 0.0024
Epoch 6/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0013 - val_loss: 0.0019
Epoch 7/2000
46/46 [==============================] - 0s 6ms/step - loss: 0.0010 - val_loss: 0.0016
Epoch 8/2000
46/46 [==============================] - 0s 6ms/step - loss: 8.5598e-04 - val_loss: 0.0016
Epoch 9/2000
46/46 [==============================] - 0s 6ms/step - loss: 7.4198e-04 - val_loss: 0.0016
Epoch 10/2000
46/46 [==============================] - 0s 6ms/step - loss: 6.6242e-04 - val

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_18\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_18\assets


Epoch 1/2000
46/46 [==============================] - 2s 9ms/step - loss: 0.0315 - val_loss: 0.0108
Epoch 2/2000
46/46 [==============================] - 0s 7ms/step - loss: 0.0020 - val_loss: 0.0083
Epoch 3/2000
46/46 [==============================] - 0s 8ms/step - loss: 0.0013 - val_loss: 0.0056
Epoch 4/2000
46/46 [==============================] - 0s 8ms/step - loss: 9.7704e-04 - val_loss: 0.0045
Epoch 5/2000
46/46 [==============================] - 0s 8ms/step - loss: 7.7350e-04 - val_loss: 0.0037
Epoch 6/2000
46/46 [==============================] - 0s 8ms/step - loss: 6.4714e-04 - val_loss: 0.0031
Epoch 7/2000
46/46 [==============================] - 0s 8ms/step - loss: 5.7606e-04 - val_loss: 0.0029
Epoch 8/2000
46/46 [==============================] - 0s 8ms/step - loss: 5.3207e-04 - val_loss: 0.0027
Epoch 9/2000
46/46 [==============================] - 0s 7ms/step - loss: 5.0138e-04 - val_loss: 0.0024
Epoch 10/2000
46/46 [==============================] - 0s 8ms/step - loss: 4

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_19\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_19\assets


In [13]:
rnn_memory = 100

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_100 = []
trained_models_rnn_100_history = []

for i in range(0,20):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_rnn_100.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_100[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_100[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_100[i].add(kr.layers.Dense(1))
    trained_models_rnn_100[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_100_history.append(trained_models_rnn_100[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_100[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_'+str(i))

(11590, 100, 14)
(11590,)
(2823, 100, 14)
(2823,)
11690
11590
11690
11590
(11590, 100, 14)
Epoch 1/2000
46/46 [==============================] - 2s 19ms/step - loss: 0.0346 - val_loss: 0.0246
Epoch 2/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0054 - val_loss: 0.0134
Epoch 3/2000
46/46 [==============================] - 1s 13ms/step - loss: 0.0026 - val_loss: 0.0073
Epoch 4/2000
46/46 [==============================] - 1s 13ms/step - loss: 0.0017 - val_loss: 0.0051
Epoch 5/2000
46/46 [==============================] - 1s 13ms/step - loss: 0.0013 - val_loss: 0.0037
Epoch 6/2000
46/46 [==============================] - 1s 13ms/step - loss: 0.0010 - val_loss: 0.0037
Epoch 7/2000
46/46 [==============================] - 1s 13ms/step - loss: 9.3093e-04 - val_loss: 0.0031
Epoch 8/2000
46/46 [==============================] - 1s 13ms/step - loss: 8.4746e-04 - val_loss: 0.0031
Epoch 9/2000
46/46 [==============================] - 1s 13ms/step - loss: 7.8255e-04 - val_l

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
46/46 [==============================] - 2s 19ms/step - loss: 0.0234 - val_loss: 0.0139
Epoch 2/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0042 - val_loss: 0.0068
Epoch 3/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0026 - val_loss: 0.0038
Epoch 4/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 5/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0015 - val_loss: 0.0021
Epoch 6/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0012 - val_loss: 0.0016
Epoch 7/2000
46/46 [==============================] - 1s 15ms/step - loss: 9.7162e-04 - val_loss: 0.0014
Epoch 8/2000
46/46 [==============================] - 1s 15ms/step - loss: 8.1537e-04 - val_loss: 0.0014
Epoch 9/2000
46/46 [==============================] - 1s 15ms/step - loss: 7.1409e-04 - val_loss: 0.0013
Epoch 10/2000
46/46 [==============================] - 1s 15ms/step - loss: 6.2

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_18\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_18\assets


Epoch 1/2000
46/46 [==============================] - 2s 19ms/step - loss: 0.0333 - val_loss: 0.0130
Epoch 2/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0021 - val_loss: 0.0089
Epoch 3/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0014 - val_loss: 0.0066
Epoch 4/2000
46/46 [==============================] - 1s 15ms/step - loss: 0.0010 - val_loss: 0.0050
Epoch 5/2000
46/46 [==============================] - 1s 15ms/step - loss: 8.2737e-04 - val_loss: 0.0046
Epoch 6/2000
46/46 [==============================] - 1s 15ms/step - loss: 6.8756e-04 - val_loss: 0.0038
Epoch 7/2000
46/46 [==============================] - 1s 15ms/step - loss: 6.0536e-04 - val_loss: 0.0031
Epoch 8/2000
46/46 [==============================] - 1s 15ms/step - loss: 5.5874e-04 - val_loss: 0.0031
Epoch 9/2000
46/46 [==============================] - 1s 15ms/step - loss: 5.2545e-04 - val_loss: 0.0027
Epoch 10/2000
46/46 [==============================] - 1s 15ms/step - l

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_19\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_19\assets


In [14]:
rnn_memory = 200

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_200 = []
trained_models_rnn_200_history = []

for i in range(0,20):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_rnn_200.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_200[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_200[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_200[i].add(kr.layers.Dense(1))
    trained_models_rnn_200[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_200_history.append(trained_models_rnn_200[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_200[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_'+str(i))

(11490, 200, 14)
(11490,)
(2723, 200, 14)
(2723,)
11690
11490
11690
11490
(11490, 200, 14)
Epoch 1/2000
45/45 [==============================] - 2s 34ms/step - loss: 0.0345 - val_loss: 0.0229
Epoch 2/2000
45/45 [==============================] - 1s 26ms/step - loss: 0.0054 - val_loss: 0.0111
Epoch 3/2000
45/45 [==============================] - 1s 30ms/step - loss: 0.0025 - val_loss: 0.0064
Epoch 4/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0016 - val_loss: 0.0044
Epoch 5/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0012 - val_loss: 0.0038
Epoch 6/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0010 - val_loss: 0.0030
Epoch 7/2000
45/45 [==============================] - 1s 31ms/step - loss: 9.2253e-04 - val_loss: 0.0029
Epoch 8/2000
45/45 [==============================] - 1s 31ms/step - loss: 8.4760e-04 - val_loss: 0.0031
Epoch 9/2000
45/45 [==============================] - 1s 31ms/step - loss: 7.8603e-04 - val_l

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
45/45 [==============================] - 3s 35ms/step - loss: 0.0231 - val_loss: 0.0157
Epoch 2/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0042 - val_loss: 0.0072
Epoch 3/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0026 - val_loss: 0.0044
Epoch 4/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0020 - val_loss: 0.0028
Epoch 5/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0016 - val_loss: 0.0020
Epoch 6/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0012 - val_loss: 0.0016
Epoch 7/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0010 - val_loss: 0.0013
Epoch 8/2000
45/45 [==============================] - 1s 31ms/step - loss: 8.4698e-04 - val_loss: 0.0012
Epoch 9/2000
45/45 [==============================] - 1s 31ms/step - loss: 7.2261e-04 - val_loss: 0.0013
Epoch 10/2000
45/45 [==============================] - 1s 31ms/step - loss: 6.3179e

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_18\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_18\assets


Epoch 1/2000
45/45 [==============================] - 3s 35ms/step - loss: 0.0330 - val_loss: 0.0124
Epoch 2/2000
45/45 [==============================] - 1s 32ms/step - loss: 0.0020 - val_loss: 0.0086
Epoch 3/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0014 - val_loss: 0.0064
Epoch 4/2000
45/45 [==============================] - 1s 31ms/step - loss: 0.0010 - val_loss: 0.0049
Epoch 5/2000
45/45 [==============================] - 1s 31ms/step - loss: 8.2007e-04 - val_loss: 0.0043
Epoch 6/2000
45/45 [==============================] - 1s 31ms/step - loss: 6.8087e-04 - val_loss: 0.0036
Epoch 7/2000
45/45 [==============================] - 1s 31ms/step - loss: 6.0490e-04 - val_loss: 0.0031
Epoch 8/2000
45/45 [==============================] - 1s 31ms/step - loss: 5.5186e-04 - val_loss: 0.0027
Epoch 9/2000
45/45 [==============================] - 1s 31ms/step - loss: 5.1716e-04 - val_loss: 0.0030
Epoch 10/2000
45/45 [==============================] - 1s 32ms/step - l

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_19\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_19\assets


In [15]:
rnn_memory = 400

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_400 = []
trained_models_rnn_400_history = []

for i in range(0,10):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_rnn_400.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_400[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_400[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_400[i].add(kr.layers.Dense(1))
    trained_models_rnn_400[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_400_history.append(trained_models_rnn_400[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_400[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_'+str(i))

(11290, 400, 14)
(11290,)
(2523, 400, 14)
(2523,)
11690
11290
11690
11290
(11290, 400, 14)
Epoch 1/2000
45/45 [==============================] - 4s 67ms/step - loss: 0.0357 - val_loss: 0.0219
Epoch 2/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0056 - val_loss: 0.0116
Epoch 3/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0026 - val_loss: 0.0069
Epoch 4/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0017 - val_loss: 0.0047
Epoch 5/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0013 - val_loss: 0.0037
Epoch 6/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0011 - val_loss: 0.0034
Epoch 7/2000
45/45 [==============================] - 3s 62ms/step - loss: 9.5232e-04 - val_loss: 0.0030
Epoch 8/2000
45/45 [==============================] - 3s 63ms/step - loss: 8.7232e-04 - val_loss: 0.0029
Epoch 9/2000
45/45 [==============================] - 3s 64ms/step - loss: 8.0638e-04 - val_l

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
45/45 [==============================] - 4s 67ms/step - loss: 0.0230 - val_loss: 0.0154
Epoch 2/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0043 - val_loss: 0.0057
Epoch 3/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0027 - val_loss: 0.0041
Epoch 4/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0020 - val_loss: 0.0025
Epoch 5/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0015 - val_loss: 0.0020
Epoch 6/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0012 - val_loss: 0.0015
Epoch 7/2000
45/45 [==============================] - 3s 63ms/step - loss: 9.7877e-04 - val_loss: 0.0013
Epoch 8/2000
45/45 [==============================] - 3s 63ms/step - loss: 8.2279e-04 - val_loss: 0.0015
Epoch 9/2000
45/45 [==============================] - 3s 63ms/step - loss: 7.1907e-04 - val_loss: 0.0013
Epoch 10/2000
45/45 [==============================] - 3s 64ms/step - loss: 6.4

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_8\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_8\assets


Epoch 1/2000
45/45 [==============================] - 4s 66ms/step - loss: 0.0160 - val_loss: 0.0427
Epoch 2/2000
45/45 [==============================] - 3s 62ms/step - loss: 0.0052 - val_loss: 0.0244
Epoch 3/2000
45/45 [==============================] - 3s 62ms/step - loss: 0.0029 - val_loss: 0.0142
Epoch 4/2000
45/45 [==============================] - 3s 63ms/step - loss: 0.0015 - val_loss: 0.0110
Epoch 5/2000
45/45 [==============================] - 3s 62ms/step - loss: 0.0010 - val_loss: 0.0090
Epoch 6/2000
45/45 [==============================] - 3s 62ms/step - loss: 7.7465e-04 - val_loss: 0.0079
Epoch 7/2000
45/45 [==============================] - 3s 62ms/step - loss: 6.8428e-04 - val_loss: 0.0072
Epoch 8/2000
45/45 [==============================] - 3s 62ms/step - loss: 6.1235e-04 - val_loss: 0.0064
Epoch 9/2000
45/45 [==============================] - 3s 63ms/step - loss: 5.6923e-04 - val_loss: 0.0060
Epoch 10/2000
45/45 [==============================] - 3s 63ms/step - loss:

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_9\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_9\assets


In [16]:
rnn_memory = 800

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_800 = []
trained_models_rnn_800_history = []

for i in range(0,5):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=24)
    
    trained_models_rnn_800.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_800[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_800[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_800[i].add(kr.layers.Dense(1))
    trained_models_rnn_800[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_800_history.append(trained_models_rnn_800[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_800[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_'+str(i))

(10890, 800, 14)
(10890,)
(2123, 800, 14)
(2123,)
11690
10890
11690
10890
(10890, 800, 14)
Epoch 1/2000
43/43 [==============================] - 7s 131ms/step - loss: 0.0368 - val_loss: 0.0206
Epoch 2/2000
43/43 [==============================] - 5s 128ms/step - loss: 0.0059 - val_loss: 0.0115
Epoch 3/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0027 - val_loss: 0.0072
Epoch 4/2000
43/43 [==============================] - 5s 127ms/step - loss: 0.0018 - val_loss: 0.0050
Epoch 5/2000
43/43 [==============================] - 5s 127ms/step - loss: 0.0013 - val_loss: 0.0038
Epoch 6/2000
43/43 [==============================] - 5s 126ms/step - loss: 0.0011 - val_loss: 0.0036
Epoch 7/2000
43/43 [==============================] - 5s 127ms/step - loss: 9.8723e-04 - val_loss: 0.0031
Epoch 8/2000
43/43 [==============================] - 5s 127ms/step - loss: 9.0836e-04 - val_loss: 0.0032
Epoch 9/2000
43/43 [==============================] - 6s 129ms/step - loss: 8.3945e-0

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
43/43 [==============================] - 7s 132ms/step - loss: 0.0239 - val_loss: 0.0171
Epoch 2/2000
43/43 [==============================] - 5s 126ms/step - loss: 0.0045 - val_loss: 0.0063
Epoch 3/2000
43/43 [==============================] - 5s 126ms/step - loss: 0.0028 - val_loss: 0.0044
Epoch 4/2000
43/43 [==============================] - 5s 126ms/step - loss: 0.0021 - val_loss: 0.0027
Epoch 5/2000
43/43 [==============================] - 5s 127ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 6/2000
43/43 [==============================] - 5s 126ms/step - loss: 0.0013 - val_loss: 0.0015
Epoch 7/2000
43/43 [==============================] - 5s 126ms/step - loss: 0.0010 - val_loss: 0.0015
Epoch 8/2000
43/43 [==============================] - 5s 127ms/step - loss: 8.7996e-04 - val_loss: 0.0013
Epoch 9/2000
43/43 [==============================] - 5s 126ms/step - loss: 7.4663e-04 - val_loss: 0.0015
Epoch 10/2000
43/43 [==============================] - 5s 127ms/step - los

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_3\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_3\assets


Epoch 1/2000
43/43 [==============================] - 7s 136ms/step - loss: 0.1354 - val_loss: 0.0495
Epoch 2/2000
43/43 [==============================] - 6s 130ms/step - loss: 0.0103 - val_loss: 0.0302
Epoch 3/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0074 - val_loss: 0.0312
Epoch 4/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0058 - val_loss: 0.0284
Epoch 5/2000
43/43 [==============================] - 6s 128ms/step - loss: 0.0047 - val_loss: 0.0249
Epoch 6/2000
43/43 [==============================] - 6s 130ms/step - loss: 0.0039 - val_loss: 0.0232
Epoch 7/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0033 - val_loss: 0.0210
Epoch 8/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0028 - val_loss: 0.0197
Epoch 9/2000
43/43 [==============================] - 6s 129ms/step - loss: 0.0024 - val_loss: 0.0168
Epoch 10/2000
43/43 [==============================] - 6s 130ms/step - loss: 0.002

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_4\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_4\assets


In [17]:
rnn_memory = 1000

trainInputX, trainInputY = sf.prepareData(trainX, trainY, rnn_memory)
testInputX, testInputY = sf.prepareData(testX, testY, rnn_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_rnn_1000 = []
trained_models_rnn_1000_history = []

for i in range(0,5):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=24)
    
    trained_models_rnn_1000.append(kr.Sequential())
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_1000[i].add(kr.layers.SimpleRNN(14,return_sequences = True))
    trained_models_rnn_1000[i].add(kr.layers.SimpleRNN(8))
    # rnn_model.add(kr.layers.SimpleRNN(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # rnn_model.add(kr.layers.BatchNormalization())
    trained_models_rnn_1000[i].add(kr.layers.Dense(1))
    trained_models_rnn_1000[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_rnn_1000_history.append(trained_models_rnn_1000[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY), callbacks=[callback]))
    
    trained_models_rnn_1000[i].save('C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_'+str(i))

(10690, 1000, 14)
(10690,)
(1923, 1000, 14)
(1923,)
11690
10690
11690
10690
(10690, 1000, 14)
Epoch 1/2000
42/42 [==============================] - 9s 179ms/step - loss: 0.0366 - val_loss: 0.0216
Epoch 2/2000
42/42 [==============================] - 7s 172ms/step - loss: 0.0060 - val_loss: 0.0126
Epoch 3/2000
42/42 [==============================] - 7s 173ms/step - loss: 0.0028 - val_loss: 0.0081
Epoch 4/2000
42/42 [==============================] - 7s 174ms/step - loss: 0.0018 - val_loss: 0.0054
Epoch 5/2000
42/42 [==============================] - 7s 173ms/step - loss: 0.0014 - val_loss: 0.0042
Epoch 6/2000
42/42 [==============================] - 7s 175ms/step - loss: 0.0011 - val_loss: 0.0038
Epoch 7/2000
42/42 [==============================] - 7s 174ms/step - loss: 0.0010 - val_loss: 0.0038
Epoch 8/2000
42/42 [==============================] - 7s 172ms/step - loss: 9.2247e-04 - val_loss: 0.0037
Epoch 9/2000
42/42 [==============================] - 7s 174ms/step - loss: 8.5468e-04

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
42/42 [==============================] - 8s 177ms/step - loss: 0.0252 - val_loss: 0.0184
Epoch 2/2000
42/42 [==============================] - 7s 173ms/step - loss: 0.0047 - val_loss: 0.0076
Epoch 3/2000
42/42 [==============================] - 7s 172ms/step - loss: 0.0029 - val_loss: 0.0042
Epoch 4/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0022 - val_loss: 0.0030
Epoch 5/2000
42/42 [==============================] - 7s 173ms/step - loss: 0.0017 - val_loss: 0.0022
Epoch 6/2000
42/42 [==============================] - 7s 172ms/step - loss: 0.0014 - val_loss: 0.0017
Epoch 7/2000
42/42 [==============================] - 7s 173ms/step - loss: 0.0011 - val_loss: 0.0014
Epoch 8/2000
42/42 [==============================] - 7s 172ms/step - loss: 9.4497e-04 - val_loss: 0.0014
Epoch 9/2000
42/42 [==============================] - 7s 171ms/step - loss: 8.0959e-04 - val_loss: 0.0014
Epoch 10/2000
42/42 [==============================] - 7s 172ms/step - los

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_3\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_3\assets


Epoch 1/2000
42/42 [==============================] - 8s 175ms/step - loss: 0.1335 - val_loss: 0.0536
Epoch 2/2000
42/42 [==============================] - 7s 170ms/step - loss: 0.0106 - val_loss: 0.0343
Epoch 3/2000
42/42 [==============================] - 7s 172ms/step - loss: 0.0076 - val_loss: 0.0316
Epoch 4/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0060 - val_loss: 0.0310
Epoch 5/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0049 - val_loss: 0.0265
Epoch 6/2000
42/42 [==============================] - 7s 169ms/step - loss: 0.0040 - val_loss: 0.0238
Epoch 7/2000
42/42 [==============================] - 7s 172ms/step - loss: 0.0034 - val_loss: 0.0219
Epoch 8/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0029 - val_loss: 0.0208
Epoch 9/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.0025 - val_loss: 0.0176
Epoch 10/2000
42/42 [==============================] - 7s 171ms/step - loss: 0.002

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_4\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/RNN_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_4\assets
